In [ ]:
# 400 meter coordinate accuracy
# 1923 earliest year
import pandas as pd

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
species_path = input("Enter the file path: ")

species = pd.read_parquet(species_path)
# species = pd.read_json(species_path)

In [ ]:
species

In [ ]:
ecoregions_path = input("Enter the file path: ")

# ecoregions = pd.read_csv(ecoregions_path)
ecoregions = pd.read_json(ecoregions_path)

In [ ]:
ecoregions

In [ ]:
new_ecoregions = pd.read_csv(
    "/media/muskrat/T7 Shield/eco_data/v3/db_exports/ecoregions.csv"
)

new_ecoregions

In [ ]:
final_ecoregions = pd.merge(new_ecoregions, ecoregions, on="unique_id", how="left")

final_ecoregions

In [ ]:
# drop columns _id, name_y, TYPE_y, coordinates_y

final_ecoregions.drop(
    columns=["_id", "name_y", "TYPE_y", "coordinates_y"], inplace=True
)
# rename columns name_x to name, TYPE_x to TYPE, coordinates_x to coordinates
final_ecoregions = final_ecoregions.rename(
    columns={"name_x": "name", "TYPE_x": "TYPE", "coordinates_x": "coordinates"}
)

final_ecoregions

In [ ]:
ecoregions = final_ecoregions

ecoregions

In [ ]:
# reset index of species and ecoregions
species.reset_index(drop=True, inplace=True)
ecoregions.reset_index(drop=True, inplace=True)

In [ ]:
# drop species_count and _id column from ecoregions

ecoregions = ecoregions.drop(["species_count", "_id"], axis=1)

ecoregions

In [ ]:
# rename column observed_ecoregion to unique_id in species

species = species.rename(columns={"observed_ecoregions": "unique_id"})

species

In [ ]:
rankedArray = []
for i in range(len(species)):
    result = species.loc[i]
    rankedResult = {}
    for curr in result["unique_id"]:
        rankedResult[curr] = rankedResult.get(curr, 0) + 1
    rankedArray.append(rankedResult)

rankedObject = {}
for item in rankedArray:
    for key in item:
        if key not in rankedObject:
            rankedObject[key] = item[key]
        else:
            rankedObject[key] += item[key]

In [ ]:
ranked_df = pd.DataFrame.from_dict(
    rankedObject, orient="index", columns=["species_count"]
)

ranked_df["unique_id"] = ranked_df.index

# drop row where unique_id = '497' LOOK INTO WHY!!!(might be ranked array logic bug) LOOKED INTO - THERE IS NO ECOREGION WITH THAT ID
ranked_df = ranked_df[ranked_df["unique_id"] != "497"]

In [ ]:
ecoregions["unique_id"] = ecoregions["unique_id"].astype(str)

ecoregions = pd.merge(
    ecoregions, ranked_df, on="unique_id", how="outer", indicator=True
)

In [ ]:
# replace nan with 0 in species_count column
ecoregions["species_count"] = ecoregions["species_count"].fillna(0)

# create a column for ranked_df called species_rank that ranks species by count, 1 being the highest
# ecoregions["species_rank"] = ecoregions["species_count"].rank(
#     ascending=False, method="first"
# )

In [ ]:
ecoregions

In [ ]:
# ecoregions.drop(["_merge", "_id"], axis=1, inplace=True)
ecoregions.drop(["_merge"], axis=1, inplace=True)

# drop row where unique_id = '<NA>'
ecoregions = ecoregions[ecoregions["unique_id"] != "<NA>"]

ecoregions

In [ ]:
# drop index from ecoregions
ecoregions.reset_index(drop=True, inplace=True)

In [ ]:
# convert species_count column to int
ecoregions["species_count"] = ecoregions["species_count"].astype(int)

In [ ]:
ecoregions

In [ ]:
# REMOVE QUOTATION MARKS AROUND COORDINATES. EASIER TO DO IN TEXT EDITOR MAYBE

In [ ]:
df_path = input("Enter the file path: ")
ecoregions.to_json(df_path, orient="records", force_ascii=False)